In [1]:
%load_ext autoreload
%autoreload 2
from domino.data.imagenet import get_imagenet_dp

#data_dp = get_imagenet_dp()
data_dp = get_imagenet_dp.out(6121).load()

In [5]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:

list(wn.synset("snowmobile.n.01").closure(lambda s: s.hypernyms()))

/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('instrumentality.n.03') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):


[Synset('tracked_vehicle.n.01'),
 Synset('self-propelled_vehicle.n.01'),
 Synset('wheeled_vehicle.n.01'),
 Synset('container.n.01'),
 Synset('vehicle.n.01'),
 Synset('instrumentality.n.03'),
 Synset('conveyance.n.03'),
 Synset('artifact.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [16]:
synsets = set(data_dp["synset"].unique())

In [17]:
hypernyms = []
for synset in synsets:
    synset =  wn.synset_from_pos_and_offset(
        pos=synset[0], offset=int(synset[1:])
    )
    for hypernym in synset.closure(lambda s: s.hypernyms()):
        hypernyms.append( {"synset": synset.name(), "hypernym": hypernym.name()})


/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 10
  for synset in acyclic_breadth_first(self, rel, depth):
/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('artifact.n.01') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 8
  for synset in acyclic_breadth_first(self, rel, depth):
/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('instrumentality.n.03') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/home/common/envs/conda/envs/domino/lib/python3

In [11]:
import pandas as pd
df = pd.DataFrame(hypernyms)

In [18]:
df.hypernym.value_counts()[20:40]

conveyance.n.03                72
vehicle.n.01                   67
commodity.n.01                 65
consumer_goods.n.01            64
hunting_dog.n.01               63
invertebrate.n.01              61
structure.n.01                 59
bird.n.01                      59
matter.n.03                    52
clothing.n.01                  51
arthropod.n.01                 47
wheeled_vehicle.n.01           45
equipment.n.01                 43
implement.n.01                 39
reptile.n.01                   36
self-propelled_vehicle.n.01    32
musical_instrument.n.01        32
diapsid.n.01                   31
working_dog.n.01               30
food.n.02                      28
Name: hypernym, dtype: int64

In [38]:
df[df["hypernym"] == "vehicle.n.01"]

,synset,hypernym
96,balloon.n.01,vehicle.n.01
127,shopping_cart.n.01,vehicle.n.01
146,warplane.n.01,vehicle.n.01
236,lifeboat.n.01,vehicle.n.01
256,freight_car.n.01,vehicle.n.01
...,...,...
11139,pickup.n.01,vehicle.n.01
11467,racer.n.02,vehicle.n.01
11606,go-kart.n.01,vehicle.n.01
11618,trimaran.n.01,vehicle.n.01


In [ ]:
df[df["hypernym"] == "vehicle.n.01"]

In [2]:
from domino.slices.imagenet import ImageNetSliceBuilder

In [3]:
sb = ImageNetSliceBuilder()

In [16]:
sb.build_rare_slice(
    data_dp, 
    target_synset="vehicle.n.01", 
    slice_synsets="balloon.n.01",
    target_frac=0.5,
    slice_frac=0.001,
    n=40_000
)

ValueError: 'a' cannot be empty unless no samples are taken

In [25]:
ImageNetSliceBuilder.build_rare_slice.__qualname__

'ImageNetSliceBuilder.build_rare_slice'

NameError: name 'dp' is not defined

In [34]:
import meerkat as mk
import numpy as np
dp = mk.DataPanel({
    "a": np.zeros((10, 100))
})

In [35]:
dp["a"][:, 10]

,(NumpyArrayColumn)
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
